# Assignment 5
The script should not give an error. Any mistake will be consider as 0.


In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
!pip install selenium
!pip install webdriver-manager

In [3]:
# this library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
from selenium.webdriver.common.by import By
import pandas as pd # for table import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [4]:
pwd

'C:\\Users\\DDelgado\\Documents\\GitHub\\Diplomado_PUCP\\Lecture_7\\Assignment_5'

In [53]:
# Set driver

service = Service(executable_path="../chromedriver-win64/chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
url = 'https://infogob.jne.gob.pe/Eleccion'
driver.get(url)

In [67]:
# We set the dictionary to compile dfs
all_tables = {}

# We then set a loop to look for each table and turn it into a df and store it in the dictioary 

for periodo in range(26, 1, -1):
    time.sleep(4)
    # Proceso para llegar a la tabla
    proceso = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
    proceso.click()
    time.sleep(3)

    presidencial = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    presidencial.click()
    time.sleep(3)

    eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    eleccion.click()
    time.sleep(3)

    # Seleccionar el periodo actual
    periodo_xpath = f'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{periodo}]'
    periodo_element = driver.find_element(By.XPATH, periodo_xpath)
    
    periodo_element.click()
    time.sleep(3)

    # Hacer clic en el botón de buscar
    buscar = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    buscar.click()
    time.sleep(3)

    # Periodo proceso
    #periodo_path = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[1]/h3/text()').text
    #periodo_value = periodo_path.get_attribute( 'innerHTML' )
    
    # Hacer clic en la pestaña de resultados
    resultados = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    resultados.click()
    time.sleep(3)

    # Obtener el HTML de la tabla
    table_path = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]')  # Añade el XPATH de la tabla aquí
    table_html = table_path.get_attribute('innerHTML')
    time.sleep(3)
    
    # Read table
    table = pd.read_html( table_html )
    table[0]
    all_tables[periodo] = table[0] 
    
    #table[0]["Periodo"] = periodo_value
    time.sleep(3)
    
    if periodo != 1:
        # We go back to search for the remaining tables
        back = driver.find_element(By.XPATH , '/html/body/div[1]/section/div[1]/a[2]')
        back.click()


InvalidSelectorException: Message: invalid selector: The result of the xpath expression "/html/body/div[1]/section/div[2]/div[1]/h3/text()" is: [object Text]. It should be an element.
  (Session info: chrome=120.0.6099.218); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00007FF648522142+3514994]
	(No symbol) [0x00007FF648140CE2]
	(No symbol) [0x00007FF647FE76AA]
	(No symbol) [0x00007FF647FECFC0]
	(No symbol) [0x00007FF647FEF38F]
	(No symbol) [0x00007FF647FEF480]
	(No symbol) [0x00007FF648031624]
	(No symbol) [0x00007FF64803197C]
	(No symbol) [0x00007FF648074EE7]
	(No symbol) [0x00007FF64805602F]
	(No symbol) [0x00007FF6480728F6]
	(No symbol) [0x00007FF648055D93]
	(No symbol) [0x00007FF648024BDC]
	(No symbol) [0x00007FF648025C64]
	GetHandleVerifier [0x00007FF64854E16B+3695259]
	GetHandleVerifier [0x00007FF6485A6737+4057191]
	GetHandleVerifier [0x00007FF64

In [ ]:
# Append datasets
all_tables_df = pd.concat(all_tables.values(), axis=0, ignore_index=True)

all_tables_df



### Siguientes pasos:
- Definir variable que capture la elección
- Abordar casos con más de una tabla (varios partidos en eleccion que no entran en una sola tabla y se dividen por paginas)
- Recortar dataframe y mantener solo variables de interés

In [ ]:
#Run packages

# Create a dataframe to collect the results (out of the for loop)
results = pd.DataFrame()

#### Possibly for loop starts here

# Open driver

service = Service(executable_path="../chromedriver-win64/chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Enter in the website
url = 'https://infogob.jne.gob.pe/Eleccion'
driver.get(url)
time.sleep(5)

# Open first selection box **(relevant in the for loop)**
proceso = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
proceso.click()
time.sleep(3)

# Select "PROCESO PRESIDENCIAL"
presidencial = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
presidencial.click()
time.sleep(3)

# Open second box
eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
eleccion.click()
time.sleep(3)

# EXAMPLE: Select first option
periodo_element = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[2]')
periodo_element.click()

# get text
elecciones_name = periodo_element.text
time.sleep(1)

# Click in search box
buscar = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
buscar.click()
time.sleep(3)

# Click "Resultados"
resultados = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
resultados.click()
time.sleep(3)

# Get table
raw_table = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]" )
raw_table_html = raw_table.get_attribute( 'innerHTML' )

# Organize the table
tabla = pd.read_html(raw_table_html)[0]
tabla_final = tabla[['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]  #keep the most important columns
tabla_final.insert(0,'Elecciones', elecciones_name)            #insert the name of round of election in the fisrt column

# Append the table in the results dataframe **(relevant in the for loop)**
results = pd.concat([results, tabla_final], ignore_index=True)


######## For loop ends here

# After finishing the loop, export the result table as an Excel file

results.to_excel('Results_group3.xlsx', index=False)